# Imports

In [1]:
import random
import sys

import importlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, metrics

import data_visualisation as dv
import data_augmentation as da
import models.resnet as resnet

2024-05-26 17:39:43.643312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 17:39:44.131131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-05-26 17:39:44.131187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-05-26 17:39:44.131193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# Load the Data

In [2]:
# load data
thismodule = sys.modules[__name__]

with np.load('data/PTB_XL_HB_1s_window.npz', allow_pickle=True) as data:
    for k in data.keys():
        if 'text' in k:
            setattr(thismodule, k, data[k])
        else:
            setattr(thismodule, k, data[k].astype(float))

# Auto-encoder Model

In [3]:
# define an lstm autoencoder model
# Define the LSTM-based autoencoder model
class LSTMAutoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = layers.LSTM(latent_dim, activation='relu', return_sequences=False, input_shape=(100, 12))
        self.repeat_vector = layers.RepeatVector(100)
        self.decoder = layers.LSTM(12, activation='relu', return_sequences=True)

    def call(self, inputs):
        encoded = self.encoder(inputs)
        repeated = self.repeat_vector(encoded)
        decoded = self.decoder(repeated)
        return decoded

In [4]:
def build_model(input_shape, latent_dim=64):
    model = LSTMAutoencoder(latent_dim)
    input = keras.Input(shape=input_shape)
    output = model(input)
    model = keras.Model(inputs=input, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    return model

In [3]:

model = build_model(X_train.shape[1:], latent_dim=128)

NameError: name 'build_model' is not defined

In [15]:
def mask_ecg(ecg, mask_ratio=0.1):
    block_size = 20
    
    for lead in range(ecg.shape[1]):
        for i in range(0, ecg.shape[0], block_size):
            if random.random() < mask_ratio:
                ecg[i:i+block_size, lead] = 0
    return ecg

def mask_lead(ecg, mask_ratio=0.1):

    for lead in range(ecg.shape[1]):
        if random.random() < mask_ratio:
            ecg[:, lead] = 0
    return ecg

In [4]:
def mask_ecg(ecg, mask_ratio=0.1, block_size=20):
    ecg_shape = tf.shape(ecg)
    num_blocks = ecg_shape[0] // block_size
    
    mask = tf.random.uniform([num_blocks, ecg_shape[1]], minval=0, maxval=1) < mask_ratio
    mask = tf.reshape(tf.tile(mask[:, None, :], [1, block_size, 1]), ecg_shape)
    
    return tf.where(mask, tf.zeros_like(ecg), ecg)

def mask_lead(ecg, mask_ratio=0.1):
    ecg_shape = tf.shape(ecg)
    
    mask = tf.random.uniform([ecg_shape[1]], minval=0, maxval=1) < mask_ratio
    mask = tf.tile(mask[None, :], [ecg_shape[0], 1])
    
    return tf.where(mask, tf.zeros_like(ecg), ecg)

In [5]:
def augment_ecg_signal(signal):
    drifted_signal, _ = da.add_random_baseline_drift(signal, strength_range=(1.5,2.5), drift_wavelength_range=(300,500))
    noised_drifted_signal = da.add_random_noise(drifted_signal, (0, 0.2))
    res = mask_lead(noised_drifted_signal, mask_ratio=0.2)
    return res

In [6]:
def train_autoencoder(model, data, epochs=50, batch_size=32):
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.MeanSquaredError()
    
    dataset = tf.data.Dataset.from_tensor_slices(X_train)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
    
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        steps = len(X_train) // batch_size
        for step, batch in enumerate(dataset):
            augmented_batch = tf.map_fn(augment_ecg_signal, batch)

            with tf.GradientTape() as tape:
                reconstructed = model(augmented_batch, training=True)
                loss = loss_fn(batch, reconstructed)
            
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            print(f"\rEpoch {epoch+1}/{epochs}, Step {step}/{steps}, Loss: {loss.numpy():.4f}", end='')
        model.save(f'model-weights/autoencoder_{epoch+1}.h5')
        
        # print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.numpy()}')

In [8]:
# train
train_autoencoder(model, X_train, epochs=5, batch_size=32)

Epoch 1/5


2024-05-26 17:41:27.412887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-26 17:41:27.835492: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x308be070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-26 17:41:27.835533: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-05-26 17:41:27.839364: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-26 17:41:27.915934: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1/5, Step 7179/7179, Loss: 0.0412Epoch 2/5
Epoch 2/5, Step 7179/7179, Loss: 0.0305Epoch 3/5
Epoch 3/5, Step 7179/7179, Loss: 0.0386Epoch 4/5
Epoch 4/5, Step 7179/7179, Loss: 0.0373Epoch 5/5
Epoch 5/5, Step 7179/7179, Loss: 0.032844384

In [9]:
modelk = keras.models.load_model('model-weights/autoencoder_1.h5')

In [7]:
def make_model(input_shape, latent_dim=64):
    inputs = keras.Input(shape=input_shape)
    x = layers.LSTM(64, activation='relu', return_sequences=False, name='encoder')(inputs)
    x = layers.RepeatVector(100, name='repeat')(x)
    x = layers.LSTM(12, activation='relu', return_sequences=True, name='decoder')(x)
    return keras.Model(inputs=inputs, outputs=x)
    
model = make_model(X_train.shape[1:], latent_dim=128)
model.compile(optimizer='adam', loss='mse')
model.summary()
    

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 12)]         0         
                                                                 
 encoder (LSTM)              (None, 64)                19712     
                                                                 
 repeat (RepeatVector)       (None, 100, 64)           0         
                                                                 
 decoder (LSTM)              (None, 100, 12)           3696      
                                                                 
Total params: 23,408
Trainable params: 23,408
Non-trainable params: 0
_________________________________________________________________


2024-05-26 17:41:15.894558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-26 17:41:15.961982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-26 17:41:15.962250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-26 17:41:15.963005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF